In [6]:
import pandas as pd
import statsmodels.api as sm
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression

# 데이터 불러오기
data = pd.read_csv("C:\\Users\\eun01\\OneDrive - 한국외국어대학교\\climates_means.csv")

# 데이터 구조 확인
data.head()

,연도,mt1,mt2,mt3,mt4,mt5,mt6,mt7,mt8,mt9,...,land4,land5,land6,land7,land8,land9,land10,land11,land12,yield
0,2013,13.606452,5.429032,-3.658065,-3.793548,-0.900000,5.006452,9.480645,17.006452,22.183871,...,-1.483871,1.019355,7.054839,11.964516,20.132258,26.464516,27.490323,29.777419,23.203226,488.9677
1,2014,14.419355,5.906452,-0.138710,-0.951613,1.364516,6.945161,12.858065,17.832258,21.858065,...,-0.235484,2.422581,8.174194,15.596774,21.464516,25.567742,27.745161,25.987097,23.435484,520.2581
2,2015,14.029032,7.767742,-2.212903,-0.877419,0.945161,5.661290,12.232258,17.935484,21.835484,...,0.009677,2.054839,7.545161,13.961290,22.293548,26.719355,27.225806,28.067742,23.900000,538.2581
3,2016,14.254839,9.012903,2.035484,-2.354839,0.519355,6.316129,13.119355,18.296774,22.325806,...,-0.400000,1.606452,7.993548,15.441935,21.287097,26.635484,28.090323,31.070968,24.154839,534.9355
4,2017,15.080645,6.729032,1.761290,-1.396774,0.132258,5.119355,13.093548,18.209677,21.506452,...,0.351613,1.735484,7.451613,15.612903,23.151613,27.493548,28.625806,27.906452,23.306452,508.0968


# RFECV

모든 행의 종속변수가 동일한지 확인

In [66]:
print(data['yield'].unique())

[488.9677 520.2581 538.2581 534.9355 508.0968 520.1613 508.0645 468.3548
 527.1935 507.7742]


특성과 종속변수 분리

In [11]:
X = data.drop(['연도', 'yield'], axis=1)
y = data['yield']

In [10]:
X

,연도,mt1,mt2,mt3,mt4,mt5,mt6,mt7,mt8,mt9,...,land3,land4,land5,land6,land7,land8,land9,land10,land11,land12
0,2013,13.606452,5.429032,-3.658065,-3.793548,-0.900000,5.006452,9.480645,17.006452,22.183871,...,-0.429032,-1.483871,1.019355,7.054839,11.964516,20.132258,26.464516,27.490323,29.777419,23.203226
1,2014,14.419355,5.906452,-0.138710,-0.951613,1.364516,6.945161,12.858065,17.832258,21.858065,...,1.258065,-0.235484,2.422581,8.174194,15.596774,21.464516,25.567742,27.745161,25.987097,23.435484
2,2015,14.029032,7.767742,-2.212903,-0.877419,0.945161,5.661290,12.232258,17.935484,21.835484,...,-0.041935,0.009677,2.054839,7.545161,13.961290,22.293548,26.719355,27.225806,28.067742,23.900000
3,2016,14.254839,9.012903,2.035484,-2.354839,0.519355,6.316129,13.119355,18.296774,22.325806,...,2.661290,-0.400000,1.606452,7.993548,15.441935,21.287097,26.635484,28.090323,31.070968,24.154839
4,2017,15.080645,6.729032,1.761290,-1.396774,0.132258,5.119355,13.093548,18.209677,21.506452,...,2.496774,0.351613,1.735484,7.451613,15.612903,23.151613,27.493548,28.625806,27.906452,23.306452
5,2018,14.548387,5.419355,-1.451613,-3.667742,-1.722581,7.409677,12.454839,17.425806,22.054839,...,-0.109677,-1.229032,-0.125806,8.406452,14.325806,20.154839,26.958065,30.903226,31.867742,22.561290
6,2019,11.964516,7.141935,-0.432258,-1.319355,0.870968,6.332258,11.270968,18.116129,21.196774,...,0.551613,-0.664516,1.964516,7.506452,13.467742,22.596774,25.887097,27.845161,28.687097,23.251613
7,2020,15.087097,7.400000,1.641935,1.561290,2.519355,7.083871,10.109677,17.196774,22.819355,...,2.167742,1.796774,3.606452,8.738710,13.883871,20.677419,27.435484,25.600000,28.367742,22.625806
8,2021,13.016129,7.612903,-0.541935,-2.280645,2.470968,8.058065,13.025806,16.387097,21.767742,...,0.764516,-0.341935,3.358065,9.577419,15.641935,19.341935,25.377419,30.554839,28.116129,23.774194
9,2022,14.712903,7.632258,0.793548,-2.203226,-1.135484,6.909677,13.345161,17.648387,22.454839,...,1.625806,-0.716129,1.054839,8.035484,16.496774,23.393548,26.348387,29.158065,27.335484,23.541935


In [13]:
y.head()

0    488.9677
1    520.2581
2    538.2581
3    534.9355
4    508.0968
Name: yield, dtype: float64

변수 표준화 & 표준화된 데이터프레임으로 변환 (다시 상수를 추가하기 위해 데이터프레임으로 변환)

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

In [15]:
X_scaled

,mt1,mt2,mt3,mt4,mt5,mt6,mt7,mt8,mt9,mt10,...,land3,land4,land5,land6,land7,land8,land9,land10,land11,land12
0,-0.501521,-1.448973,-1.953422,-1.412518,-1.026975,-1.569781,-2.028507,-1.041442,0.409292,0.371506,...,-1.430175,-1.376816,-0.809393,-1.426372,-2.074826,-1.004833,-0.035416,-0.557486,0.638432,-0.355531
1,0.374316,-1.010070,0.046375,0.531294,0.626549,0.489679,0.588035,0.394256,-0.317219,-0.441164,...,0.153525,0.064428,0.526290,0.180612,0.742653,0.011567,-1.348186,-0.387047,-1.646545,0.123837
2,-0.046225,0.701058,-1.132241,0.582041,0.320341,-0.874157,0.103212,0.573719,-0.367572,-0.902409,...,-1.066802,0.347463,0.176249,-0.722448,-0.525963,0.644048,0.337637,-0.734397,-0.392238,1.082571
3,0.197063,1.845764,1.281813,-0.428477,0.009422,-0.178532,0.790461,1.201837,0.725792,-0.074050,...,1.470752,-0.125504,-0.250556,-0.078728,0.622548,-0.123789,0.214860,-0.156200,1.418241,1.608543
4,1.086803,-0.253852,1.126009,0.226815,-0.273232,-1.449846,0.770468,1.050415,-1.101277,0.820201,...,1.316318,0.742222,-0.127734,-0.856750,0.755164,1.298678,1.470963,0.201938,-0.489471,-0.142479
5,0.513338,-1.457869,-0.699654,-1.326470,-1.627614,0.983127,0.275649,-0.312377,0.121565,1.190452,...,-1.130392,-1.082608,-1.899434,0.514050,-0.243214,-0.987606,0.687079,1.725100,1.898573,-1.680448
6,-2.270574,0.125740,-0.120428,0.279768,0.266166,-0.161399,-0.641515,0.887778,-1.791823,-0.434888,...,-0.509630,-0.430883,0.090274,-0.778021,-0.908800,0.875383,-0.880689,-0.320166,-0.018863,-0.255663
7,1.093754,0.362985,1.058188,2.250057,1.469799,0.637028,-1.541186,-0.710559,1.826350,-2.248492,...,1.007451,2.410638,1.653177,0.991051,-0.586016,-0.588921,1.385964,-1.821754,-0.211385,-1.547291
8,-1.137546,0.558711,-0.182750,-0.377731,1.434468,1.671898,0.717987,-2.118216,-0.518629,0.992776,...,-0.309775,-0.058469,1.416746,2.195131,0.777684,-1.607781,-1.626795,1.492095,-0.363068,0.822914
9,0.690591,0.576505,0.576110,-0.324778,-1.198923,0.451985,0.965396,0.074589,1.013520,0.726069,...,0.498729,-0.490470,-0.775618,-0.018524,1.440768,1.483254,-0.205415,0.557918,-0.833676,0.343547


변수들에 상수 추가

In [16]:
X_scaled = sm.add_constant(X_scaled)

선형 회귀 모델 생성

In [17]:
model = sm.OLS(y, X_scaled).fit()

변수 선택을 위한 RFECV 설정 & 모델 피팅

In [67]:
lr = LinearRegression()
rfecv = RFECV(estimator=lr, step=1, cv=5)

model1 = rfecv.fit(X_scaled, y)

In [51]:
for i in range(X_scaled.shape[1]):
    print('Colum: %d, Selected %s, Rank: %.3f' % (i, rfecv.support_[i], rfecv.ranking_[i]))

Colum: 0, Selected False, Rank: 95.000
Colum: 1, Selected False, Rank: 91.000
Colum: 2, Selected False, Rank: 14.000
Colum: 3, Selected False, Rank: 60.000
Colum: 4, Selected False, Rank: 81.000
Colum: 5, Selected False, Rank: 59.000
Colum: 6, Selected False, Rank: 45.000
Colum: 7, Selected True, Rank: 1.000
Colum: 8, Selected False, Rank: 32.000
Colum: 9, Selected False, Rank: 48.000
Colum: 10, Selected False, Rank: 78.000
Colum: 11, Selected False, Rank: 33.000
Colum: 12, Selected False, Rank: 77.000
Colum: 13, Selected False, Rank: 39.000
Colum: 14, Selected False, Rank: 28.000
Colum: 15, Selected False, Rank: 80.000
Colum: 16, Selected False, Rank: 55.000
Colum: 17, Selected False, Rank: 58.000
Colum: 18, Selected False, Rank: 19.000
Colum: 19, Selected True, Rank: 1.000
Colum: 20, Selected False, Rank: 46.000
Colum: 21, Selected False, Rank: 74.000
Colum: 22, Selected False, Rank: 29.000
Colum: 23, Selected False, Rank: 87.000
Colum: 24, Selected False, Rank: 5.000
Colum: 25, Sele

In [58]:
model1.n_features_

3

In [49]:
X_scaled.columns[model1.support_]

Index(['mt7', 'ht7', 'sun7'], dtype='object')

### 선택된 변수

In [59]:
selected_features = X_scaled.columns[rfecv.support_]
selected_features

Index(['mt7', 'ht7', 'sun7'], dtype='object')

선택된 변수로 회귀 모델 다시 생성

In [21]:
final_model = sm.OLS(y, X_scaled[selected_features]).fit()

print(final_model.summary())

                                 OLS Regression Results                                
Dep. Variable:                  yield   R-squared (uncentered):                   0.001
Model:                            OLS   Adj. R-squared (uncentered):             -0.426
Method:                 Least Squares   F-statistic:                           0.003435
Date:                Fri, 24 May 2024   Prob (F-statistic):                        1.00
Time:                        19:02:15   Log-Likelihood:                         -76.577
No. Observations:                  10   AIC:                                      159.2
Df Residuals:                       7   BIC:                                      160.1
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

C:\Users\eun01\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


# RFE

선형 회귀 모델 생성

In [72]:
estimator = LinearRegression()

RFE를 이용하여 변수 선택 (여기서는 8개의 변수를 선택)

In [73]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator, n_features_to_select=8, step=1)
rfe.fit(X_scaled, y)

RFE(estimator=LinearRegression(), n_features_to_select=8)

선택된 변수 확인

In [74]:
selected_features2 = rfe.support_
selected_indices = [i for i, x in enumerate(selected_features2) if x]
print("선택된 변수의 인덱스:", selected_indices)

선택된 변수의 인덱스: [7, 19, 24, 59, 60, 79, 82, 86]


In [75]:
selected_features2

array([False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False,  True, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False])

### 선택된 변수들

In [76]:
selected_features2 = X_scaled.columns[rfe.support_]
selected_features2

Index(['mt7', 'ht7', 'ht12', 'ra11', 'ra12', 'sun7', 'sun10', 'land2'], dtype='object')

선택된 변수로 회귀 모델 다시 생성

In [77]:
final_model2 = sm.OLS(y, X_scaled[selected_features2]).fit()

print(final_model2.summary())

                                 OLS Regression Results                                
Dep. Variable:                  yield   R-squared (uncentered):                   0.002
Model:                            OLS   Adj. R-squared (uncentered):             -3.992
Method:                 Least Squares   F-statistic:                          0.0003873
Date:                Fri, 24 May 2024   Prob (F-statistic):                        1.00
Time:                        19:30:23   Log-Likelihood:                         -76.577
No. Observations:                  10   AIC:                                      169.2
Df Residuals:                       2   BIC:                                      171.6
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

C:\Users\eun01\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


# 결론

### 1. RFECV : 4월의 평균기온, 4월의 평균최고기온, 9월의 평균최고기온

### 2. RFE : 4월의 평균기온, 4월의 평균최고기온, 9월의 평균최고기온, 8월의 강수량, 9월의 강수량, 4월의 합계일조시간, 7월의 합계일조시간, 11월의 평균지면온도

* RFECV : 최적의 변수 개수와 해당 변수를 자동으로 추출
* RFE : 추출되는 변수 개수 임의로 설정(8개)